In [1]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
#pyqtgraph -> fast plotting
import pyqtgraph as pg
from pyqtgraph.Qt import QtGui
%gui qt5
import time
from scipy.spatial.distance import euclidean

In [101]:
class Graph(object):

    def __init__(self, graphDict=None):
        """ initializes a graph object 
            If no dictionary or None is given, 
            an empty dictionary will be used
        """
        if graphDict == None:
            graphDict = {}
        self.graphDict = graphDict

    def getVertices(self):
        """ returns the vertices of a graph """
        return list(self.graphDict.keys())

    def getEdges(self):
        """ returns the edges of a graph """
        return self.generateEdges()

    def addVertex(self, vertex):
        """ If the vertex "vertex" is not in 
            self.graphDict, a key "vertex" with an empty
            list as a value is added to the dictionary. 
            Otherwise nothing has to be done. 
        """
        if vertex not in self.graphDict:
            self.graphDict[vertex] = []

    def addEdge(self, edge):
        """  
            between two vertices can be multiple edges! 
        """
        edge = set(edge)
        (vertex1, vertex2) = tuple(edge)
        if vertex1 in self.graphDict:
            self.graphDict[vertex1].append(vertex2)
        else:
            self.graphDict[vertex1] = [vertex2]

    def generateEdges(self):
        """ A static method generating the edges of the 
            graph "graph". Edges are represented as sets 
            with one (a loop back to the vertex) or two 
            vertices 
        """
        edges = []
        for vertex in self.graphDict:
            for neighbour in self.graphDict[vertex]:
                if {neighbour, vertex} not in edges:
                    edges.append({vertex, neighbour})
        return edges

    def __str__(self):
        res = "vertices: "
        for k in self.graphDict:
            res += str(k) + " "
        res += "\nedges: "
        for edge in self.generateEdges():
            res += str(edge) + " "
        return res

In [231]:
class Stack:
     def __init__(self):
         self.items = []

     def isEmpty(self):
         return self.items == []

     def push(self, item):
         self.items.append(item)

     def pop(self):
         return self.items.pop()

     def peek(self):
         return self.items[len(self.items)-1]

     def size(self):
         return len(self.items)

### Load Data

In [207]:
#one person slow walk
#TLV Data Load
tlvData = (loadmat('C:\\Users\\hasna\\Documents\\GitHub\\OccupancyDetection\\Data\\Matlab Data\\slowWalk.mat'))['tlvStream'][0]

In [6]:
#two people slow walk
#TLV Data Load
#loads of points
tlvData = (loadmat('C:\\Users\\hasna\\Documents\\GitHub\\OccupancyDetection\\Data\\Matlab Data\\2PeopleMoving.mat'))['tlvStream'][0]

In [3]:
#three people walk
#TLV Data Load
tlvData = (loadmat('C:\\Users\\hasna\\Documents\\GitHub\\OccupancyDetection\\Data\\Matlab Data\\3PeopleWalking.mat'))['tlvStream'][0]

In [147]:
#initialise variables
lostSync = False

#valid header variables and constant
magicBytes = np.array([2,1,4,3,6,5,8,7], dtype= 'uint8')

isMagicOk = False
isDataOk = False
gotHeader = False

frameHeaderLength = 52 #52 bytes long
tlvHeaderLengthInBytes = 8
pointLengthInBytes = 16
frameNumber = 1
targetFrameNumber = 0
targetLengthInBytes = 68

#graph constraints
weightThreshold = 0.2 #minimum distance between points
minClusterSize = 5

### Plotting Setup

In [ ]:
app = QtGui.QApplication([])
pg.setConfigOption('background','w')

In [ ]:
win = pg.GraphicsWindow(title="Testing GUI")
plot1 = win.addPlot()
plot1.setXRange(-6,6)
plot1.setYRange(0,6)
plot1.setLabel('left',text = 'Y position (m)')
plot1.setLabel('bottom', text= 'X position (m)')
s1 = plot1.plot([],[],pen=None,symbol='o')

plot2 = win.addPlot()
plot2.setXRange(-6,6)
plot2.setYRange(0,6)
plot2.setLabel('left',text = 'Y position (m)')
plot2.setLabel('bottom', text= 'X position (m)')
s2 = plot2.plot([],[],pen=None,symbol='o')

### Main Code

In [11]:
for tlvStream in tlvData:
    tlvStream = np.frombuffer(tlvStream, dtype = 'uint8')
    
    #tlv header
    index = 0
    #tlv header parsing
    tlvType = tlvStream[index:index+4].view(dtype=np.uint32)
    tlvLength = tlvStream[index+4:index+8].view(dtype=np.uint32)
    
    index += tlvHeaderLengthInBytes
    tlvDataLength = tlvLength - tlvHeaderLengthInBytes

    if tlvType == 6: 
        numberOfPoints = tlvDataLength/pointLengthInBytes
        p = tlvStream[index:index+tlvDataLength[0]].view(np.single)
        pointCloud = np.reshape(p,(4, int(numberOfPoints)),order="F")

        if not(pointCloud is None):
            #constrain point cloud to within the effective sensor range
            #range 1 < x < 6
            #azimuth -50 deg to 50 deg
            #check whether corresponding range and azimuth data are within the constraints

            effectivePointCloud = np.array([])
            for index in range(0, len(pointCloud[0,:])):
                if (pointCloud[0,index] > 1 and pointCloud[0,index] < 6) \
                and (pointCloud[1, index] > -50*np.pi/180 \
                     and pointCloud[1, index] < 50*np.pi/180):
        
                    #concatenate columns to the new point cloud
                    if len(effectivePointCloud) == 0:
                        effectivePointCloud = np.reshape(pointCloud[:, index], (4,1), order="F")
                    else:
                        point = np.reshape(pointCloud[:, index], (4,1),order="F")
                        effectivePointCloud = np.hstack((effectivePointCloud, point))
                        

            if len(effectivePointCloud) != 0:
                posX = np.multiply(effectivePointCloud[0,:], np.sin(effectivePointCloud[1,:]))
                posY = np.multiply(effectivePointCloud[0,:], np.cos(effectivePointCloud[1,:]))
                #vertex dataframe
                vertexID = np.arange(len(posX))
                vertexDf = pd.DataFrame({'VertexID':vertexID, 'X':posX, 'Y':posY})
                #minimum number of points to qualify as a person
                if len(vertexDf.values) > 5:
                    #preallocate edgeMatrix
                    #edges are denoted by their respective weights
                    #undirected graph with constraint that two nodes can only be connected by one edge
                    edgeMatrix = np.zeros((len(posX), len(posY)))
                    #vertices are saved as np arrays
                    #evaluate edge Matrix and create graph
                    for rowIndex in range(0, edgeMatrix.shape[0]):
                        for colIndex in range(0, edgeMatrix.shape[1]):
                            if rowIndex == colIndex:
                                continue #diagonal element
                            elif edgeMatrix[rowIndex, colIndex] != 0:
                                continue #element already filled
                            else:
                                pointA = vertexDf.values[rowIndex][1:] #x,y point
                                pointB = vertexDf.values[colIndex][1:] #x,y point
                                length = euclidean(pointA, pointB)
                                #fill elements
                                edgeMatrix[rowIndex,colIndex] = length
                                edgeMatrix[colIndex, rowIndex] = length
                                
                    #weight based reduction of graph/remove edges by replacing edge weight by np.NaN
                    weightMask = np.logical_or(np.greater(edgeMatrix,weightThreshold), np.equal(edgeMatrix, 0))
                    edgeMatrix[weightMask] = np.NaN
                    
                    break
                    
                    #perform DFS to find connected components
                    componentsList = list()
                    visitedSet = set()
                    for vertex in vertexDf['VertexID']:
                        if vertex == 0: #very first iteration
                            visitedNodes = dfs(vertexDf, edgeMatrix, vertex)
                            componentsList = visitedNodes
                            visitedSet.add(visitedNodes)
                        elif vertex not in visitedSet:
                            visitedNodes = dfs(vertexDf, edgeMatrix, vertex)
                            componentsList = visitedNodes
                            visitedSet.add(visitedNodes)
                else:
                    print('NOT ENOUGH POINTS')

            
        break

In [365]:
def dfs(vertexDf, edgeMatrix, startNode):
    
    #initialize
    visited = set()
    dfsStack = set()
    
    #dfs
    while not(len(dfsStack) == 0):
        vertex = dfsStack.pop()
        if vertex not in visited:
            visited.add(vertex)
            #find next nodes to visit
            #nodes must be connected to the current visited vertex and not in the visited list
            #push those nodes to the stack
            unvisitedNodes = edgeMatrix[vertex, not(np.isnan(edgeMatrix[vertex, :]))]
            for node in unvisitedNodes:
                dfsStack.add(node)
    return visited

### Main code setup for unit testing

In [3]:
#dfs unit test and overall algorithm test set 
posX = np.array([2,2.1,2.2,2.1,2.2,2.2,5,5.1,5,4,5])
posY = np.array([4,4,4,3.8,3.9,3.95,2,2,2.1,4,5])

In [347]:
#unit test to check vertex and edge generation
posX = np.array([1,1,1,2,3])
posY = np.array([4,3,2,4,4])

In [150]:
#vertex dataframe
vertexID = np.arange(len(posX))
vertexDf = pd.DataFrame({'VertexID':vertexID, 'X':posX, 'Y':posY})
#minimum number of points to qualify as a person
if len(vertexDf.values) >= 5:
    #preallocate edgeMatrix
    #edges are denoted by their respective weights
    #undirected graph with constraint that two nodes can only be connected by one edge
    edgeMatrix = np.zeros((len(posX), len(posY)))
    #vertices are saved as np arrays
    #evaluate edge Matrix and create graph
    for rowIndex in range(0, edgeMatrix.shape[0]):
        for colIndex in range(0, edgeMatrix.shape[1]):
            if rowIndex == colIndex:
                continue #diagonal element
            elif edgeMatrix[rowIndex, colIndex] != 0:
                continue #element already filled
            else:
                pointA = (vertexDf.values[rowIndex])[1:] #extract x y position disregarding the vertexID
                pointB = (vertexDf.values[colIndex])[1:] #extract x y position disregarding the vertexID
                length = euclidean(pointA, pointB)
                #fill elements
                edgeMatrix[rowIndex,colIndex] = length
                edgeMatrix[colIndex, rowIndex] = length

    #weight based reduction of graph/remove edges by replacing edge weight by np.NaN
    weightMask = np.logical_or(np.greater(edgeMatrix,weightThreshold), np.equal(edgeMatrix, 0))
    edgeMatrix[weightMask] = np.NaN

    #perform DFS to find connected components
    componentsList = list() #list of components
    vertexList = list() #used to hold vertices that have been considered
    for vertex in vertexDf['VertexID']:
        if vertex not in vertexList:
            visitedNodes = dfs(vertexDf, edgeMatrix, vertex)
            componentsList.append(visitedNodes)
            for vertex in visitedNodes:
                if vertex not in vertexList:
                    vertexList.append(vertex)
                    
    #use minimum cluster size to remove bad cluster matches
    clusterList = np.array([])
    for cluster in componentsList:
        if len(cluster) >= minClusterSize:
            clusterList = np.append(clusterList, np.array(cluster))
else:
    print('NOT ENOUGH POINTS')


NameError: name 'componentList' is not defined

<class 'list'>
<class 'numpy.ndarray'>


In [131]:
def dfs(vertexDf, edgeMatrix, startNode):
    
    visited = []
    dfsStack = [int(startNode)]

    while not(len(dfsStack) == 0):
        vertex = dfsStack.pop()
        if vertex not in visited:
            #find unvisited nodes
            unvisitedNodes = vertexDf['VertexID'].values[np.logical_not(np.isnan(edgeMatrix[int(vertex), :]))]
            unvisitedNodes = list(unvisitedNodes)
            visited.append(vertex)
            #add unvisited nodes to the stack
            for node in unvisitedNodes:
                if node not in visited:
                    dfsStack.append(node)
    return visited